In [18]:
from keys import client_id
from keys import client_secret
from keys import user_name
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

scope = 'user-library-read playlist-modify-public playlist-read-private user-read-recently-played app-remote-control user-top-read'

redirect_uri = 'https://developer.spotify.com/dashboard/applications/0743a195f7654b5ab95560a95e89316a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_name, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("NO TOKEN FOUND")

In [33]:
import pandas as pd 

user_top_tracks = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
song_data = user_top_tracks["items"]

song_ids = []
song_names = []

for i in range(0, len(song_data)):
    if song_data[i]['id'] != None:
        song_ids.append(song_data[i]['id'])
        song_names.append(song_data[i]['name'])

song_features = []
for i in range(0, len(song_ids)):
    features = sp.audio_features(song_ids[i])
    for song in features:
        song_features.append(song)

top_tracks_df = pd.DataFrame(features, index=song_names)
top_tracks_df = top_tracks_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]


In [35]:
top_tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
HOLIDAY,15jRg8Ejs7X3JEHw38TSN3,0.167,0.69,193059,0.578,0,6,0.305,-7.436,1,0.0929,99.961,0.416
Death of a Bachelor,15jRg8Ejs7X3JEHw38TSN3,0.167,0.69,193059,0.578,0,6,0.305,-7.436,1,0.0929,99.961,0.416
Daylight,15jRg8Ejs7X3JEHw38TSN3,0.167,0.69,193059,0.578,0,6,0.305,-7.436,1,0.0929,99.961,0.416
Knock Knock,15jRg8Ejs7X3JEHw38TSN3,0.167,0.69,193059,0.578,0,6,0.305,-7.436,1,0.0929,99.961,0.416
Don't Matter,15jRg8Ejs7X3JEHw38TSN3,0.167,0.69,193059,0.578,0,6,0.305,-7.436,1,0.0929,99.961,0.416


In [41]:
#Vectorizes the song names
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 6), max_features=10000)
X_names_sparse = vectorizer.fit_transform(song_names)
X_names_sparse.shape


(50, 394)